In [1]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)
import matplotlib.pyplot as plt

# from folium import LayerControl
from eo_tools_dev.util import show_insar_coh, show_insar_phi, show_sar_int
import geopandas as gpd
from eodag import EODataAccessGateway

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:eodag.config:Loading user configuration from: /data/eodag_config.yml
INFO:eodag.core:usgs: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:aws_eos: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:meteoblue: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:hydroweb_next: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:wekeo: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:creodias_s3: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:Locations configuration loaded from /root/.config/eodag/locations.yml


# Set up parameters and output dir

## Important
- The products must be **unzipped** either manually or by using `extract=True` in the download call. 
- In the following products, only IW1 and IW2 are relevant as IW3 contains only water pixels.

In [3]:
# change to your custom locations
data_dir = "/data/S1"

# ids = [
#  "S1A_IW_SLC__1SDV_20230904T063730_20230904T063757_050174_0609E3_DAA1", 
#  "S1A_IW_SLC__1SDV_20230916T063730_20230916T063757_050349_060FCD_6814"
# ]
# primary_dir = f"{data_dir}/{ids[0]}.SAFE"
# secondary_dir = f"{data_dir}/{ids[1]}.SAFE"
outputs_prefix="/data/res/test-zipfile"

# Download S-1 products

In [9]:
# load a geometry
file_aoi = "../data/Etna.geojson"
shp = gpd.read_file(file_aoi).geometry[0]

search_criteria = {
    "productType": "S1_SAR_SLC",
    "start": "2018-12-22",
    "end": "2018-12-29",
    "geom": shp
}

results, _ = dag.search(**search_criteria)
# to_dl = [it for it in results if it.properties["id"] in ids]
# print(f"{len(to_dl)} products to download")
# dag.download_all(to_dl, outputs_prefix="/data/S1/", extract=False)

INFO:eodag.core:Searching product type 'S1_SAR_SLC' on provider: cop_dataspace
INFO:eodag.search.qssearch:Sending search request: http://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel1/search.json?startDate=2018-12-22&completionDate=2018-12-29&geometry=POLYGON ((14.9140 37.7885, 14.9140 37.6992, 15.0785 37.6992, 15.0785 37.7885, 14.9140 37.7885))&productType=SLC&maxRecords=20&page=1&exactCount=1


INFO:eodag.core:Found 6 result(s) on provider 'cop_dataspace'


In [10]:
from eo_tools.util import explore_products
explore_products(results, shp)

# Pre-process InSAR pair

In [ ]:
from eo_tools.S1.process import process_insar

out_dir = process_insar(
    dir_prm=primary_dir,
    dir_sec=secondary_dir,
    outputs_prefix=outputs_prefix,
    aoi_name=None,
    shp=shp,
    pol="vv",
    subswaths=["IW1", "IW2", "IW3"],
    write_coherence=True,
    write_interferogram=True,
    write_primary_amplitude=True,
    write_secondary_amplitude=False,
    apply_fast_esd=True,
    dem_upsampling=1.8,
    dem_force_download=False,
    dem_buffer_arc_sec=40,
    boxcar_coherence=[3, 10],
    filter_ifg=True,
    multilook=[1, 4],
    warp_kernel="bicubic",
    clip_to_shape=True,
)

In [ ]:
show_insar_coh(f"{out_dir}/coh_vv.tif")

In [ ]:
show_insar_phi(f"{out_dir}/phi_vv.tif")

In [ ]:
show_sar_int(f"{out_dir}/amp_prm_vv.tif")